In [8]:
#!pip install pandas numpy
#!/usr/bin/env python3

import pandas as pd
import numpy as np
import re

In [9]:
df = pd.read_csv(r"C:\Users\GK111\Downloads\audible_uncleaned.csv\audible_uncleaned.csv")
df.head()


,name,author,narrator,time,releasedate,language,stars,price
0,Geronimo Stilton #11 & #12,Writtenby:GeronimoStilton,Narratedby:BillLobely,2 hrs and 20 mins,04-08-08,English,5 out of 5 stars34 ratings,468.00
1,The Burning Maze,Writtenby:RickRiordan,Narratedby:RobbieDaymond,13 hrs and 8 mins,01-05-18,English,4.5 out of 5 stars41 ratings,820.00
2,The Deep End,Writtenby:JeffKinney,Narratedby:DanRussell,2 hrs and 3 mins,06-11-20,English,4.5 out of 5 stars38 ratings,410.00
3,Daughter of the Deep,Writtenby:RickRiordan,Narratedby:SoneelaNankani,11 hrs and 16 mins,05-10-21,English,4.5 out of 5 stars12 ratings,615.00
4,"The Lightning Thief: Percy Jackson, Book 1",Writtenby:RickRiordan,Narratedby:JesseBernstein,10 hrs,13-01-10,English,4.5 out of 5 stars181 ratings,820.00


In [10]:
df.sample(10)

,name,author,narrator,time,releasedate,language,stars,price
84443,Der Atlantis-Komplex,Writtenby:EoinColfer,Narratedby:RufusBeck,7 hrs and 42 mins,11-03-11,german,Not rated yet,334.00
6430,North Dakota,"Writtenby:BridgetParker,TylerMaine",Narratedby:anonymous,12 mins,02-03-22,English,Not rated yet,93.00
19070,Io sono Sofonisba,Writtenby:HaiderBucar,Narratedby:ValentinaVeratrini,1 hr and 50 mins,19-03-20,italian,Not rated yet,190.00
79994,The Book of Martyrs,"Writtenby:DanieWare,AlecWorley,PhilKelly","Narratedby:RemmieMilner,HelenMcAlpine,GraceAnd...",11 hrs,29-01-22,English,Not rated yet,"1,138.00"
72105,The Last Bookshop,Writtenby:EmmaYoung,Narratedby:OliviaBeardsley,10 hrs and 21 mins,01-01-22,English,Not rated yet,785.00
13444,"Learn Czech: Level 1 - Introduction to Czech, ...",Writtenby:InnovativeLanguage.com,Narratedby:InnovativeLanguageLearing,4 hrs and 9 mins,16-01-15,English,Not rated yet,469.00
14564,Paul Cézanne Homme carré - et Artiste pointu,Writtenby:GéraldinePuireux,Narratedby:JulienAllouf,3 hrs and 12 mins,07-01-22,french,Not rated yet,566.00
41049,Gilded Suffragists,Writtenby:JohannaNeuman,Narratedby:C.S.E.Cooney,6 hrs and 59 mins,05-09-17,English,Not rated yet,668.00
727,La fée Fifolette et le gâteau Bonbon,Writtenby:MimiZagarriga,Narratedby:VanessaParachou,5 mins,16-03-22,french,Not rated yet,151.00
19472,Romancing with Life,Writtenby:DevAnand,Narratedby:DerekDenzel,13 hrs and 9 mins,01-05-20,English,4 out of 5 stars19 ratings,879.00


In [11]:
# Clean author
df['author'] = (
    df['author']
    .str.replace(r'Writtenby:', '', regex=True)   # remove "Writtenby:"
    .str.strip()                                 # remove extra spaces
)

# Clean narrator
df['narrator'] = (
    df['narrator']
    .str.replace(r'Narratedby:', '', regex=True) # remove "Narratedby:"
    .str.strip()
)

print(df['narrator'].sample(10))
print('###################')
print(df['author'].sample(10))

81490                                        KevinYoung
16965                                         BillBrown
9038                                            unknown
13266                                 RussianPod101.com
81587                                     MichaelTaylor
54745                                   ImaniJadePowers
22002                                        uncredited
84104                                      SamanthaKaye
23150    LaurieWoolever,ShepGordon,ChristopherBourdain,
6514                                      MileneLeclerc
Name: narrator, dtype: object
###################
28680                                        週刊東洋経済編集部
34862                                      JohnH.Evans
44879                                 JeroenWindmeijer
30745    KathleenHurley-editor,PriscillaShumway-editor
27141            CrystalDuffy,Dr.KennethMoise-foreword
41789                           LeonardoLiporoneBaruki
74523                                   LindsayMcKenna
22804

In [12]:
#Clean rerease date
df['releasedate'] = pd.to_datetime(
    df['releasedate'],
    errors='coerce',              # invalid dates become NaT (null datetime)
    dayfirst=True                 # important: your sample is day-month-year
)
df['releasedate'].sample(10)

C:\Users\GK111\AppData\Local\Temp\ipykernel_6736\957617465.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['releasedate'] = pd.to_datetime(


25075   2015-02-17
42992   2016-10-24
67877   2021-04-14
38302   2020-03-17
67089   2017-05-19
69880   2022-03-01
82540   2013-05-03
65279   2013-05-21
25796   2022-02-01
7384    2015-12-04
Name: releasedate, dtype: datetime64[ns]

In [13]:
# Function to convert strings like "2 hrs and 36 mins" to total minutes as int like 156
def convert_time_to_minutes(time_str):
    if pd.isna(time_str):   # Handle missing values
        return None
    
    # Extract hours and minutes using regex
    hours = re.search(r'(\d+)\s*hr', time_str)
    mins = re.search(r'(\d+)\s*min', time_str)
    
    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60   # convert hrs to mins
    if mins:
        total_minutes += int(mins.group(1))         # add mins
    
    return total_minutes


In [28]:
import re

# split stars and ratings properly
# the stars part is like "4.6 out of 5 stars"
# the ratings part is like "1,234 ratings"
# othertimes its "Not rated yet"

def extract_stars(text):
    # if its "Not rated yet" then return nothing
    if "Not rated yet" in text:
        return None
    
    # try to find something like "4.5 out of 5 stars"
    match = re.search(r"(\d+(\.\d+)?) out of 5 stars", text)
    if match:
        return float(match.group(1))  # this gives me the number before "out of 5"
    else:
        return None

def extract_ratings(text):
    # again, ignore "Not rated yet"
    if "Not rated yet" in text:
        return None
    
    # look for number before "ratings"
    # \d means digit, + means one or more, ,? allows commas
    match = re.search(r"([\d,]+) ratings", text)
    if match:
        # remove commas from numbers like "1,234"
        return int(match.group(1).replace(",", ""))
    else:
        return None

# now apply both
df["stars_clean"] = df["stars"].apply(extract_stars)
df["ratings"] = df["stars"].apply(extract_ratings)

# quick check
print(df[["stars", "stars_clean", "ratings"]].head(15))


                            stars  stars_clean  ratings
0      5 out of 5 stars34 ratings          5.0     34.0
1    4.5 out of 5 stars41 ratings          4.5     41.0
2    4.5 out of 5 stars38 ratings          4.5     38.0
3    4.5 out of 5 stars12 ratings          4.5     12.0
4   4.5 out of 5 stars181 ratings          4.5    181.0
5      5 out of 5 stars72 ratings          5.0     72.0
6      5 out of 5 stars11 ratings          5.0     11.0
7      5 out of 5 stars50 ratings          5.0     50.0
8       5 out of 5 stars5 ratings          5.0      5.0
9      5 out of 5 stars58 ratings          5.0     58.0
10  4.5 out of 5 stars130 ratings          4.5    130.0
11      5 out of 5 stars6 ratings          5.0      6.0
12      5 out of 5 stars7 ratings          5.0      7.0
13     5 out of 5 stars41 ratings          5.0     41.0
14      4 out of 5 stars2 ratings          4.0      2.0


In [29]:
# Remove duplicates and reset index
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87489 entries, 0 to 87488
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         87489 non-null  object        
 1   author       87489 non-null  object        
 2   narrator     87489 non-null  object        
 3   time         87489 non-null  object        
 4   releasedate  87489 non-null  datetime64[ns]
 5   language     87489 non-null  object        
 6   stars        87489 non-null  object        
 7   price        87489 non-null  object        
 8   stars_clean  15072 non-null  float64       
 9   ratings      9250 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 6.7+ MB


In [30]:
#get a statistical summary of the numerical columns
df.describe(include='all')

,name,author,narrator,time,releasedate,language,stars,price,stars_clean,ratings
count,87489,87489,87489,87489,87489,87489,87489,87489,15072.00000,9250.000000
unique,82767,48374,29717,2284,NaN,36,665,1011,NaN,NaN
top,The Art of War,"矢島雅弘,石橋遊",anonymous,2 mins,NaN,English,Not rated yet,586.00,NaN,NaN
freq,20,874,1034,372,NaN,61884,72417,5533,NaN,NaN
mean,NaN,NaN,NaN,NaN,2018-06-23 22:53:20.411480064,NaN,NaN,NaN,4.45694,34.587243
min,NaN,NaN,NaN,NaN,1998-12-27 00:00:00,NaN,NaN,NaN,1.00000,2.000000
25%,NaN,NaN,NaN,NaN,2016-09-06 00:00:00,NaN,NaN,NaN,4.00000,2.000000
50%,NaN,NaN,NaN,NaN,2020-02-06 00:00:00,NaN,NaN,NaN,4.50000,5.000000
75%,NaN,NaN,NaN,NaN,2021-08-17 00:00:00,NaN,NaN,NaN,5.00000,14.000000
max,NaN,NaN,NaN,NaN,2025-11-14 00:00:00,NaN,NaN,NaN,5.00000,12573.000000


In [34]:
#check for missing values
df.isnull().sum() 
#stars and ratings seem to have the most missing values

name               0
author             0
narrator           0
time               0
releasedate        0
language           0
stars              0
price              0
stars_clean    72417
ratings        78239
dtype: int64

In [32]:
#check uniqueness as duplicates
for col in df.columns:
    print(col, ":", df[col].nunique(), "unique values")

print('\n duplicated:', df.duplicated().sum())


name : 82767 unique values
author : 48374 unique values
narrator : 29717 unique values
time : 2284 unique values
releasedate : 5058 unique values
language : 36 unique values
stars : 665 unique values
price : 1011 unique values
stars_clean : 9 unique values
ratings : 383 unique values

 duplicated: 0


In [35]:
df.ratings
df.stars_clean

0        5.0
1        4.5
2        4.5
3        4.5
4        4.5
        ... 
87484    NaN
87485    NaN
87486    NaN
87487    NaN
87488    NaN
Name: stars_clean, Length: 87489, dtype: float64

In [36]:
#Do more exploration to see what cleaning is needed
# Check languages - maybe some are written funny (english vs English)
print(df["language"].unique())

# Check prices - are they strings or numeric?
print(df["price"].head())

# Convert price to numeric
df["price"] = pd.to_numeric(df["price"], errors="coerce")

# Sanity check: what's the cheapest and most expensive audiobook?
print("Price min:", df["price"].min())
print("Price max:", df["price"].max())

# Check time column again - make sure all were converted to minutes succcesfully
print(df["time"].sample(10))

# Check duplicates by book name + author (sometimes same title appears multiple times)
dupes = df.duplicated(subset=["name", "author"])
print("Duplicate books:", dupes.sum())


['English' 'Hindi' 'spanish' 'german' 'french' 'catalan' 'swedish'
 'italian' 'danish' 'finnish' 'dutch' 'hebrew' 'russian' 'polish'
 'galician' 'afrikaans' 'icelandic' 'romanian' 'japanese' 'tamil'
 'portuguese' 'urdu' 'hungarian' 'czech' 'bulgarian' 'mandarin_chinese'
 'basque' 'korean' 'arabic' 'greek' 'turkish' 'ukrainian' 'slovene'
 'norwegian' 'telugu' 'lithuanian']
0    468.00
1    820.00
2    410.00
3    615.00
4    820.00
Name: price, dtype: object
Price min: 11.0
Price max: 999.0
5085                59 mins
72051     7 hrs and 21 mins
19019    15 hrs and 56 mins
58972     4 hrs and 12 mins
43783     5 hrs and 44 mins
60868    10 hrs and 22 mins
1668                 3 mins
13310      1 hr and 44 mins
16396               16 mins
68495     9 hrs and 10 mins
Name: time, dtype: object
Duplicate books: 1523


Languages - messy casing ('English' vs 'spanish' vs 'german'). Some are lowercase, some are proper, some snake_case (mandarin_chinese).

Price - successfully numeric now , looks sane (11.0 min, 999.0 max).

Time - still a string mix ("59 mins", "7 hrs and 21 mins"). Needs parsing into total minutes.

Duplicates - 1,523 duplicates on name + author. That’s not tragic out of 87k entries, but still dirty.

In [37]:
#  Clean language column (capitalize first letter, replace snake_case)
df["language_clean"] = df["language"].str.strip().str.title() # capitalize first letter of each word
df["language_clean"] = df["language_clean"].str.replace("_", " ") # replace underscores with spaces

print(df["language_clean"].unique()) # check unique values

# Handle duplicates (drop them if exact duplicated)
df = df.drop_duplicates(subset=["name", "author"]) # drop duplicates based on name + author match
print("After dropping duplicates:", df.shape) # check new shape

# Convert time into minutes

def convert_time_to_minutes(x):
    if pd.isna(x): # Handle missing values
        return None # return None if time is missing
    hours = re.search(r"(\d+)\s*hr", x) # look for something like "2 hr" or "1 hr"
    mins = re.search(r"(\d+)\s*min", x) # look for something like "36 min"

    h = int(hours.group(1)) * 60 if hours else 0 # convert hrs to mins
    m = int(mins.group(1)) if mins else 0 # get mins
    return h + m # return total minutes

df["time_mins"] = df["time"].apply(convert_time_to_minutes) # apply function to time column

print("Time in minutes summary:")
print(df["time_mins"].describe()) # check summary stats


['English' 'Hindi' 'Spanish' 'German' 'French' 'Catalan' 'Swedish'
 'Italian' 'Danish' 'Finnish' 'Dutch' 'Hebrew' 'Russian' 'Polish'
 'Galician' 'Afrikaans' 'Icelandic' 'Romanian' 'Japanese' 'Tamil'
 'Portuguese' 'Urdu' 'Hungarian' 'Czech' 'Bulgarian' 'Mandarin Chinese'
 'Basque' 'Korean' 'Arabic' 'Greek' 'Turkish' 'Ukrainian' 'Slovene'
 'Norwegian' 'Telugu' 'Lithuanian']
After dropping duplicates: (85966, 11)
Time in minutes summary:
count    85966.000000
mean       416.873694
std        362.906468
min          1.000000
25%        142.000000
50%        386.500000
75%        584.000000
max       8595.000000
Name: time_mins, dtype: float64


### Do feature Ingineering on the dataset

In [38]:
df.columns

Index(['name', 'author', 'narrator', 'time', 'releasedate', 'language',
       'stars', 'price', 'stars_clean', 'ratings', 'language_clean',
       'time_mins'],
      dtype='object')

In [39]:
# Create column with Book length categories to show short, medium, long and very long books
#create a function to categorize length
def categorize_length(minutes):
    if minutes < 120:   # less than 2 hrs
        return "Short"
    elif minutes < 360: # less than 6 hrs
        return "Medium"
    elif minutes < 720: # less than 12 hrs
        return "Long"
    else:
        return "Very Long"

#call the function to create new column 'length_category'
df["length_category"] = df["time_mins"].apply(categorize_length)

# Release year 
df["release_year"] = pd.to_datetime(df["releasedate"]).dt.year

# create Price buckets to show cheap, mid and premium books
def price_bucket(price):
    if price < 200:
        return "Cheap"
    elif price <= 500:
        return "Mid"
    else:
        return "Premium"

#call the function to create new column 'price_bucket'

df["price_bucket"] = df["price"].astype(float).apply(price_bucket)

# Define Star rating category to show low, medium and high rated books in 3 categories in new column 'star_category'
df["star_category"] = pd.cut(df["stars_clean"],
                             bins=[0, 3, 4, 5],
                             labels=["Low", "Medium", "High"],
                             include_lowest=True)

# create column Rating density based on number of ratings per minute of audiobook length
df["rating_density"] = df["ratings"] / df["time_mins"]

# Get Author productivity based on number of books by author
author_counts = df["author"].value_counts()
df["author_books_count"] = df["author"].map(author_counts)

# Quick check
print(df[["time_mins","length_category","release_year","price","price_bucket",
          "stars_clean","star_category","ratings","rating_density","author","author_books_count"]].head(10))


   time_mins length_category  release_year  price price_bucket  stars_clean  \
0        140          Medium          2008  468.0          Mid          5.0   
1        788       Very Long          2018  820.0      Premium          4.5   
2        123          Medium          2020  410.0          Mid          4.5   
3        676            Long          2021  615.0      Premium          4.5   
4        600            Long          2010  820.0      Premium          4.5   
5        635            Long          2018  656.0      Premium          5.0   
6        143          Medium          2014  233.0          Mid          5.0   
7        752       Very Long          2017  820.0      Premium          5.0   
8        656            Long          2017    NaN      Premium          5.0   
9        802       Very Long          2019  820.0      Premium          5.0   

  star_category  ratings  rating_density           author  author_books_count  
0          High     34.0        0.242857  Geronimo

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85966 entries, 0 to 87488
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                85966 non-null  object        
 1   author              85966 non-null  object        
 2   narrator            85966 non-null  object        
 3   time                85966 non-null  object        
 4   releasedate         85966 non-null  datetime64[ns]
 5   language            85966 non-null  object        
 6   stars               85966 non-null  object        
 7   price               76673 non-null  float64       
 8   stars_clean         14657 non-null  float64       
 9   ratings             8961 non-null   float64       
 10  language_clean      85966 non-null  object        
 11  time_mins           85966 non-null  int64         
 12  length_category     85966 non-null  object        
 13  release_year        85966 non-null  int32         
